In [ ]:
# Initial settings

import locale
from google.colab import drive

locale.getpreferredencoding = lambda: 'UTF-8'

drive.mount('/content/gdrive/', force_remount=True)

In [ ]:
# Define the folder where the model will be trained

ROOT_DIR = '/content/gdrive/MyDrive/SafeSkies/model_training/'

In [ ]:
# Install the necessary libraries

!pip install ultralytics
!pip install Pillow

In [ ]:
# Create and train a YOLO model from scratch

import os
from ultralytics import YOLO

model = YOLO('yolov8s.yaml')

results = model.train(data=os.path.join(ROOT_DIR, 'config.yaml'), epochs=300, patience=30, batch=64, augment=True)

In [ ]:
# Save the model and training results

import shutil

shutil.copytree('/content/runs/detect/train', os.path.join(ROOT_DIR, 'training_results'))

In [ ]:
# Save the values of key training metrics

filename = os.path.join(ROOT_DIR, 'training_results', 'metrics.txt')

metrics = {
  'Precision': results.results_dict.get('metrics/precision(B)'),
  'Recall': results.results_dict.get('metrics/recall(B)'),
  'mAP@0.5': results.results_dict.get('metrics/mAP50(B)'),
  'mAP@[.5:.95]': results.results_dict.get('metrics/mAP50-95(B)'),
  'Fitness': results.results_dict.get('fitness'),
}

with open(filename, 'w') as file:
  for metric_name, value in metrics.items():
    if value is None:
      value = 'Not available'
    file.write(f'{metric_name}: {value}\n')

In [ ]:
# Perform testing

model = YOLO(os.path.join(ROOT_DIR, 'training_results', 'weights', 'best.pt'))

test_results = model.val(data=os.path.join(ROOT_DIR, 'test_config.yaml'))

In [ ]:
# Save the test results

shutil.copytree('/content/runs/detect/val', os.path.join(ROOT_DIR, 'test_results'))

In [ ]:
# Save the values of key test metrics

filename = os.path.join(ROOT_DIR, 'test_results', 'metrics.txt')

metrics = {
  'Precision': test_results.results_dict.get('metrics/precision(B)'),
  'Recall': test_results.results_dict.get('metrics/recall(B)'),
  'mAP@0.5': test_results.results_dict.get('metrics/mAP50(B)'),
  'mAP@[.5:.95]': test_results.results_dict.get('metrics/mAP50-95(B)'),
  'Fitness': test_results.results_dict.get('fitness'),
}

with open(filename, 'w') as file:
  for metric_name, value in metrics.items():
    if value is None:
      value = 'Not available'
    file.write(f'{metric_name}: {value}\n')

In [ ]:
# Save the model's predictions as images

from PIL import Image

VAL_IMAGES_DIR = os.path.join(ROOT_DIR, 'data', 'validation', 'images')
image_list = os.listdir(VAL_IMAGES_DIR)

VAL_PREDICTIONS_DIR = os.path.join(ROOT_DIR, 'training_results', 'predictions')
os.makedirs(VAL_PREDICTIONS_DIR, exist_ok=True)

for image in image_list:
  predictions = model.predict([os.path.join(VAL_IMAGES_DIR, image)])
  prediction = Image.fromarray(predictions[0].plot()[:,:,::-1])
  prediction.save(os.path.join(VAL_PREDICTIONS_DIR, image))


TEST_IMAGES_DIR = os.path.join(ROOT_DIR, 'data', 'test', 'images')
image_list = os.listdir(TEST_IMAGES_DIR)

TEST_PREDICTIONS_DIR = os.path.join(ROOT_DIR, 'test_results', 'predictions')
os.makedirs(TEST_PREDICTIONS_DIR, exist_ok=True)

for image in image_list:
  predictions = model.predict([os.path.join(TEST_IMAGES_DIR, image)])
  prediction = Image.fromarray(predictions[0].plot()[:,:,::-1])
  prediction.save(os.path.join(TEST_PREDICTIONS_DIR, image))